### For large datasets we use batch training
---

we divide the dataset into batches and we train each individual batch and calculate loss for each batch and update weights correspondingly


### some terms


---

* epoch = one forward and backward pass of **ALL** data points.
* batch_size = number of training samples in one forward & backward pass
* number of iterations = number of passes, each pass using [batch_size] number of samples

e.g. 100 samples, batch_size = 20 ⇒ 100/20 = 5 iterations for each epoch

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
import numpy as np
import math

class WineDataset(Dataset):
    def __init__(self):
        xy = np.loadtxt('/content/drive/MyDrive/dataML/wine.csv',delimiter = ',',dtype=np.float32,skiprows=1)
        self.x = torch.from_numpy(xy[:,1:])
        self.y = torch.from_numpy(xy[:,:1])
        self.n_samples = xy.shape[0]

    def __getitem__(self,index):
        return self.x[index],self.y[index]
    def __len__(self):
        return self.n_samples


dataset = WineDataset()
# here num_workers mean it will work parallely with 2 threads working together
dataloader = DataLoader(dataset = dataset, batch_size = 4, shuffle=True, num_workers = 2)

# training loop
num_epochs = 2
total_samples = len(dataset)
n_iterations = math.ceil(total_samples / 4) #num_samples/batch_size
print(total_samples, n_iterations)

for epoch in range(num_epochs):
    for i,(inputs, labels) in enumerate(dataloader):
        # forward backward and update
        if (i+1)%5 == 0:
            print(f'epoch {epoch+1}/{num_epochs}, step {i+1}/{n_iterations}, inputs {inputs.shape}')
    print('\n')


178 45
epoch 1/2, step 5/45, inputs torch.Size([4, 13])
epoch 1/2, step 10/45, inputs torch.Size([4, 13])
epoch 1/2, step 15/45, inputs torch.Size([4, 13])
epoch 1/2, step 20/45, inputs torch.Size([4, 13])
epoch 1/2, step 25/45, inputs torch.Size([4, 13])
epoch 1/2, step 30/45, inputs torch.Size([4, 13])
epoch 1/2, step 35/45, inputs torch.Size([4, 13])
epoch 1/2, step 40/45, inputs torch.Size([4, 13])
epoch 1/2, step 45/45, inputs torch.Size([2, 13])


epoch 2/2, step 5/45, inputs torch.Size([4, 13])
epoch 2/2, step 10/45, inputs torch.Size([4, 13])
epoch 2/2, step 15/45, inputs torch.Size([4, 13])
epoch 2/2, step 20/45, inputs torch.Size([4, 13])
epoch 2/2, step 25/45, inputs torch.Size([4, 13])
epoch 2/2, step 30/45, inputs torch.Size([4, 13])
epoch 2/2, step 35/45, inputs torch.Size([4, 13])
epoch 2/2, step 40/45, inputs torch.Size([4, 13])
epoch 2/2, step 45/45, inputs torch.Size([2, 13])




# Transform methods on dataset


---

convert the MNIST dataset to tensor using the following
--------
```python
    dataset = torchvision.datasets.MNIST(root = './data',transform = torchvision.transforms.ToTensor())
```
Transforms can be applied to PIL images, tensors, ndarrays, or custom data
during creation of the DataSet

complete list of built-in transforms:
https://pytorch.org/docs/stable/torchvision/transforms.html

On Images
---------
* CenterCrop, Grayscale, Pad, RandomAffine
RandomCrop, RandomHorizontalFlip, RandomRotation
Resize, Scale

On Tensors
----------
* LinearTransformation, Normalize, RandomErasing

Conversion
----------
* ToPILImage: from tensor or ndrarray
* ToTensor : from numpy.ndarray or PILImage

Generic
-------
Use Lambda

Custom
------
Write own class

Compose multiple Transforms
---------------------------
```python
composed = transforms.Compose([Rescale(256),RandomCrop(224)])
```

In [23]:
class WineDataset(Dataset):
    def __init__(self, transform):
        xy = np.loadtxt('/content/drive/MyDrive/dataML/wine.csv',delimiter = ',',dtype=np.float32,skiprows=1)
        self.x = xy[:,1:]
        self.y = xy[:,:1]
        self.n_samples = xy.shape[0]
        self.transform = transform

    def __getitem__(self,index):
        sample = self.x[index],self.y[index]
        if self.transform:
            sample = self.transform(sample)
        return sample
    def __len__(self):
        return self.n_samples

class ToTensor:
    def __call__(self,sample):
        inputs,targets = sample
        return torch.from_numpy(inputs),torch.from_numpy(targets)

class MulTransform:
    def __init__(self,factor):
        self.factor = factor

    def __call__(self,sample):
        inputs, target = sample
        inputs *= self.factor
        return inputs,target
dataset = WineDataset(transform = None)
features,lables = dataset[0]
print(features)
print(type(features), type(lables))
print('\n')
# we club up both the transforms in one
composed = torchvision.transforms.Compose([ToTensor(),MulTransform(2)])
dataset = WineDataset(transform = composed)
features,lables = dataset[0]
print(features)
print(type(features), type(lables))


[1.423e+01 1.710e+00 2.430e+00 1.560e+01 1.270e+02 2.800e+00 3.060e+00
 2.800e-01 2.290e+00 5.640e+00 1.040e+00 3.920e+00 1.065e+03]
<class 'numpy.ndarray'> <class 'numpy.ndarray'>


tensor([2.8460e+01, 3.4200e+00, 4.8600e+00, 3.1200e+01, 2.5400e+02, 5.6000e+00,
        6.1200e+00, 5.6000e-01, 4.5800e+00, 1.1280e+01, 2.0800e+00, 7.8400e+00,
        2.1300e+03])
<class 'torch.Tensor'> <class 'torch.Tensor'>
